In [ ]:
!pip install -q --upgrade keras-cv
!pip install -q --upgrade k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 61.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# Import libraries
import tensorflow as tf
from keras_cv.models import segmentation as sm
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
from google.colab import drive
# from pathlib import Path

drive.mount('/content/drive', force_remount=True)

In [ ]:
# Define paths
# base_dir = Path('/content/drive/My Drive/Dataset/dataset_ta/')  # Mounted Drive path (optional)
# base_dir = Path('path/to/your/local/dataset')  # Local dataset path
train_dir = '/content/drive/My Drive/Dataset/dataset_ta/Dataset Mapir/per-tanaman/'
val_dir = '/content/drive/My Drive/Dataset/dataset_ta/MAPIR_TEST_IMG/Coba di Lokasi'
# test_dir = base_dir / 'test'

In [ ]:
# hyperparams
img_height = 224
img_width = 224
channel = 3
epoch = 100
batch = 16

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

# Replace 'path/to/your/image.jpg' with the actual path to your image
img = load_img('/content/drive/My Drive/Dataset/dataset_ta/Dataset Mapir/per-tanaman/2024_0608/2024_0608_111357_002.JPG')

if img is not None:
    print("Image loaded successfully!")
else:
    print("Error loading image. Check path and format.")

Image loaded successfully!


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation (optional)
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [ ]:
# Define validation data generator (no augmentation)
val_datagen = ImageDataGenerator()

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch,
    class_mode="binary"
)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch,
    class_mode="binary"
)

Found 235 images belonging to 1 classes.
Found 5 images belonging to 1 classes.


In [ ]:
print(len(train_generator))

0


In [ ]:
import keras_cv
backbone = keras_cv.models.ResNet50V2Backbone(input_shape=[img_height, img_width, channel])

# Load pre-trained model (modify as needed)
model = sm.DeepLabV3Plus(backbone,num_classes=1)
# +1 to account for the polybag segmentation class

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(
    train_generator,
    epochs=epoch,
    validation_data=val_generator,
)

# Save the trained model (replace with your desired name)
# model.save("plant_segmentation_model.h5")

print("Segmentation model training complete!")

Epoch 1/50


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

In [ ]:
# Function to segment and classify images
def segment_and_classify(image_path, plant_id):
    # Load image
    img = load_image(image_path, (img_height, img_width))

    # Preprocess image (if needed)
    img = preprocess_image(img)  # Define your preprocessing function

    # Predict mask and classification
    predictions = model.predict(np.expand_dims(img, axis=0))
    mask = predictions[0, :, :, 0] > 0.5  # Threshold for polybag segmentation

    # Extract class index and convert to plant health label
    class_index = np.argmax(predictions[1, 0])
    plant_health = plant_health_map.get(str(class_index), "unknown")

    # Generate output filename based on plant ID and health
    output_filename = f"{output_dir}/{plant_id}_{plant_health}.jpg"

    # Apply mask to original image (optional)
    segmented_img = apply_mask(img, mask)  # Define your mask application function

    # Save segmented image
    save_image(segmented_img, output_filename)

# Loop through test images and segment based on plant ID
for image_path in test_dir.glob("*.jpg"):
    plant_id = image_path.stem[:3]  # Assuming plant ID is in the filename prefix
    segment_and_classify(image_path, plant_id)

# Define helper functions (load_image, preprocess_image, apply_mask, save_image)
# Implement these functions based on your specific image format and library

print("Segmentation and classification complete. Check the output directory for results.")